In [1]:
import json
import pandas as pd
import requests
from simple_salesforce import Salesforce, SalesforceLogin, SFType

In [2]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [3]:
results = sf.query_all(
    """
   select Id, IdentificationId__pc from Account 
    """
)
df = pd.DataFrame(results["records"])
df.shape
Pacientes_SF = df[["Id", "IdentificationId__pc"]]
Pacientes_SF.shape

(1540058, 2)

In [4]:
# Recuperar empresas ID de salesforce
results = sf.query_all(
    """
   select id,Name,Nombre_Empresa__c,Empresa__c, Empresa_Padre__c from Programas_por_Empresas__c
    """
)
df = pd.DataFrame(results["records"])
df.shape
Empresas_SF = df[["Id", "Nombre_Empresa__c", "Name", "Empresa__c","Empresa_Padre__c"]]
Empresas_SF.head()



,Id,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,a1W8c000007vDN8EAM,Metlife Holding Metlife Laboratorio Maver Ltda.,A-0000,0018c00002axQXrAAM,Metlife
1,a1W8c000007vDN9EAM,Metlife Holding Metlife Aguas Del Valle S.A.,A-0001,0018c00002axQXsAAM,Metlife
2,a1W8c000007vDNAEA2,Metlife Holding Metlife Berliam Spa.,A-0002,0018c00002axQXtAAM,Metlife
3,a1W8c000007vDNBEA2,Metlife Holding Metlife Besalco Maquinarias S.A.,A-0003,0018c00002axQXuAAM,Metlife
4,a1W8c000007vDNCEA2,Metlife Holding Metlife Carlos Escarate Y Cia ...,A-0004,0018c00002axQXvAAM,Metlife


In [5]:
Empresas_SF.shape

(7879, 5)

In [6]:

# Relacion pacientes y empresas
results = sf.query_all(
    """
  select Id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c 
    """
)
df = pd.DataFrame(results)
df.shape
Pacientes_empresas = pd.DataFrame(results["records"])
Pacientes_empresas = Pacientes_empresas[
    ["Paciente__c", "Programas_por_Empresas__c"]
]
Pacientes_empresas.head()

,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLbpAAE,a1W8c000007vDPvEAM
1,0018c00002amLbqAAE,a1W8c000007vDSpEAM
2,0018c00002amLbrAAE,a1W8c000007vDSpEAM
3,0018c00002amLbsAAE,a1W8c000007vDSpEAM
4,0018c00002amLc2AAE,a1W8c000007vDSpEAM


In [7]:
#une Pacientes y el que hace de union Pacientes_empresas
pacientes_y_union = pd.merge(
    left=Pacientes_SF,
    right=Pacientes_empresas,
    how="left",
    left_on="Id",
    right_on="Paciente__c",
)
pacientes_y_union.head()

,Id,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ
1,0018c00002amLUGAA2,153338949,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083j8uEAA
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI


In [8]:
pacientes_y_union.shape

(1647629, 4)

In [9]:
pacientes_y_empresas_final = pd.merge(
    left=pacientes_y_union,
    right=Empresas_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
pacientes_y_empresas_final.head()

,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,0018c00002amLTMAA2,173805004,0018c00002amLTMAA2,a1W8c0000083icPEAQ,a1W8c0000083icPEAQ,Metlife Vulco Vulco S.A.,A-1075,0018c00002cXJZeAAO,Metlife
1,0018c00002amLUGAA2,153338949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0018c00002amLUHAA2,177735213,0018c00002amLUHAA2,a1W8c0000083j8wEAA,a1W8c0000083j8wEAA,Farmacias Cruz Verde Holding Cruz Verde Farmac...,A-2322,0018c00002cXclVAAS,Farmacias Cruz Verde
3,0018c00002amLUIAA2,169332924,0018c00002amLUIAA2,a1W8c0000083j8uEAA,a1W8c0000083j8uEAA,Ey Holding Ey Ey,A-2320,0018c00002cXclTAAS,Ey
4,0018c00002amLUJAA2,172957692,0018c00002amLUJAA2,a1W8c0000083iWhEAI,a1W8c0000083iWhEAI,Minera Abra Holding Minera Abra Minera Abra,A-0721,0018c00002cXJTrAAO,Minera Abra


In [24]:
nuevos = "C:/Users/jmartinez/Documents/cargas/ver.xlsx"
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

,apellidos,nombres,damse,nada,nacimiento,derty,rut1,sexo,rut
0,Galvan Miranda,Rafael,05/07/1976,05-07-1976,1976-07-05 00:00:00,GAMR-760705-HDFLRF03,GAMR760705HDFLRF03,Masculino,GAMR760705HDFLRF03
1,Espinosa Zamudio,Maria Teresa,25/02/1970,25-02-1970,1970-02-05 00:00:00,EIZT-700225-MDFSMR07,EIZT700225MDFSMR07,Femenino,EIZT700225MDFSMR07
2,Nava Martinez,Maria De Los Angeles,18/04/1975,18-04-1975,1975-04-08 00:00:00,NAMA-750418-MDFVRN07,NAMA750418MDFVRN07,Femenino,NAMA750418MDFVRN07
3,Jimenez Raya,Judith,28/05/1978,28-05-1978,1978-05-08 00:00:00,JIRJ-780528-MMCMYD09,JIRJ780528MMCMYD09,Femenino,JIRJ780528MMCMYD09
4,Basurto Trujillo,Leonardo,06/11/1970,06-11-1970,1970-11-16 00:00:00,BATL-701106-HPLSRN02,BATL701106HPLSRN02,Masculino,BATL701106HPLSRN02


In [25]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.upper()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].astype(str)
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.strip()
Pacientes_SF["IdentificationId__pc"] = Pacientes_SF["IdentificationId__pc"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["IdentificationId__pc"])

In [26]:
Pacientes_Archivo.head()

,apellidos,nombres,damse,nada,nacimiento,derty,rut1,sexo,rut,RutEstaoNo
0,Galvan Miranda,Rafael,05/07/1976,05-07-1976,1976-07-05 00:00:00,GAMR-760705-HDFLRF03,GAMR760705HDFLRF03,Masculino,GAMR760705HDFLRF03,False
1,Espinosa Zamudio,Maria Teresa,25/02/1970,25-02-1970,1970-02-05 00:00:00,EIZT-700225-MDFSMR07,EIZT700225MDFSMR07,Femenino,EIZT700225MDFSMR07,False
2,Nava Martinez,Maria De Los Angeles,18/04/1975,18-04-1975,1975-04-08 00:00:00,NAMA-750418-MDFVRN07,NAMA750418MDFVRN07,Femenino,NAMA750418MDFVRN07,False
3,Jimenez Raya,Judith,28/05/1978,28-05-1978,1978-05-08 00:00:00,JIRJ-780528-MMCMYD09,JIRJ780528MMCMYD09,Femenino,JIRJ780528MMCMYD09,False
4,Basurto Trujillo,Leonardo,06/11/1970,06-11-1970,1970-11-16 00:00:00,BATL-701106-HPLSRN02,BATL701106HPLSRN02,Masculino,BATL701106HPLSRN02,False


In [27]:
Pacientes_con_ID = pd.merge(
    left=Pacientes_Archivo,
    right=pacientes_y_empresas_final,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
Pacientes_con_ID.head()


,apellidos,nombres,damse,nada,nacimiento,derty,rut1,sexo,rut,RutEstaoNo,Id_x,IdentificationId__pc,Paciente__c,Programas_por_Empresas__c,Id_y,Nombre_Empresa__c,Name,Empresa__c,Empresa_Padre__c
0,Galvan Miranda,Rafael,05/07/1976,05-07-1976,1976-07-05 00:00:00,GAMR-760705-HDFLRF03,GAMR760705HDFLRF03,Masculino,GAMR760705HDFLRF03,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Espinosa Zamudio,Maria Teresa,25/02/1970,25-02-1970,1970-02-05 00:00:00,EIZT-700225-MDFSMR07,EIZT700225MDFSMR07,Femenino,EIZT700225MDFSMR07,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nava Martinez,Maria De Los Angeles,18/04/1975,18-04-1975,1975-04-08 00:00:00,NAMA-750418-MDFVRN07,NAMA750418MDFVRN07,Femenino,NAMA750418MDFVRN07,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Jimenez Raya,Judith,28/05/1978,28-05-1978,1978-05-08 00:00:00,JIRJ-780528-MMCMYD09,JIRJ780528MMCMYD09,Femenino,JIRJ780528MMCMYD09,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Basurto Trujillo,Leonardo,06/11/1970,06-11-1970,1970-11-16 00:00:00,BATL-701106-HPLSRN02,BATL701106HPLSRN02,Masculino,BATL701106HPLSRN02,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
Pacientes_con_ID.to_excel(
    "C:/Users/jmartinez/Documents/cargas/Pacientes_y_empresas_nuevas.xlsx", index=False
)


In [29]:
Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]
Final_pacientes_y_empresa = Final_pacientes_y_empresa[
    Final_pacientes_y_empresa["RutEstaoNo"] == False
]

if ("nombres" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["FirstName"] = Final_pacientes_y_empresa["nombres"]
if ("apellidos" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["LastName"] = Final_pacientes_y_empresa["apellidos"]
if ("rut" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Final_pacientes_y_empresa["rut"]
if ("email" in Final_pacientes_y_empresa.columns) == True:
    Final_pacientes_y_empresa["email"] = Final_pacientes_y_empresa["email"].str.lower()
    Subir_pacientes["PersonEmail"] = Final_pacientes_y_empresa["email"]
if ("nacimiento" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Final_pacientes_y_empresa["nacimiento"]
if ("sexo" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Final_pacientes_y_empresa["sexo"]
if ("direccion" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Final_pacientes_y_empresa.columns) == True:
    Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]

Subir_pacientes["BillingCountry"] = "Chile"
Subir_pacientes["Identification_Type__c"] = "RUT"
Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"


Subir_pacientes.to_excel(
    "C:/Users/jmartinez/Documents/cargas/cargas_cuenta_SFWEB.xlsx", index=False
)
Subir_pacientes.head()


,FirstName,LastName,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingCountry,Identification_Type__c,RecordTypeId
0,Rafael,Galvan Miranda,GAMR760705HDFLRF03,Masculino,Chile,RUT,0128c000002Byp4AAC
1,Maria Teresa,Espinosa Zamudio,EIZT700225MDFSMR07,Femenino,Chile,RUT,0128c000002Byp4AAC
2,Maria De Los Angeles,Nava Martinez,NAMA750418MDFVRN07,Femenino,Chile,RUT,0128c000002Byp4AAC
3,Judith,Jimenez Raya,JIRJ780528MMCMYD09,Femenino,Chile,RUT,0128c000002Byp4AAC
4,Leonardo,Basurto Trujillo,BATL701106HPLSRN02,Masculino,Chile,RUT,0128c000002Byp4AAC
